<a href="https://colab.research.google.com/github/ariffinsarhid/Predicting-Airline-On-Time-Performance-In-Progress-/blob/main/Model_on_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action = 'ignore')
import io

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import roc_curve, RocCurveDisplay, roc_auc_score
from sklearn.metrics import plot_confusion_matrix, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline


from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.over_sampling import SMOTE

In [2]:
url_1 = 'https://raw.githubusercontent.com/ariffinsarhid/Predicting-Airline-On-Time-Performance-In-Progress-/main/Data/final_table.csv'

df = pd.read_csv(url_1, index_col='flightdate')
df = df.drop(columns= 'Unnamed: 0')

In [3]:
df = pd.get_dummies(df, columns = ['airline', 'origincity', 'destcity'])

In [4]:
df_2018 = df[(df['year'] == 2018)]
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38130 entries, 10/04/18 to 09/02/18
Columns: 721 entries, year to destcity_Yuma
dtypes: float64(4), int64(14), uint8(703)
memory usage: 31.1+ MB


In [5]:
x3 = df_2018.drop(['arrdel15'],axis= 1)
y3 = df_2018.arrdel15

#train test split
x_train, x_test, y_train, y_test = train_test_split(x3, y3, stratify=y3, random_state = 42)


print(x_train.shape, y_train.shape)

(28597, 720) (28597,)


In [6]:
y3.value_counts()

0.0    30825
1.0     7305
Name: arrdel15, dtype: int64

In [7]:
y3.shape

(38130,)

In [8]:
def score(model,x_train, x_test):
    
    """
    This function scores the model by returning the confusion matrix and classification report of the model
    
    """
    
    # Get predictions
    preds = model.predict(x_test)
    cm = confusion_matrix(y_test, preds)
    ConfusionMatrixDisplay(cm, display_labels=['Flight Not Delay','Flight Delay']).plot(cmap='GnBu')

    print("Training Score:",model.score(x_train,y_train))
    print("Testing Score:",model.score(x_test,y_test))
    print("\n")
    print(classification_report(y_test, preds)) 

In [9]:
def roc(model, model_name, x_train, x_test): 
    
    """
    This function generates the ROC/AUC curve of the model.
    
    Due to the imbalanced nature of the data, the preferred metrics for this project is ROC/AUC.
    
    """      
    
    plt.figure(figsize=(16,10))
    
    # Predict probabilities from X_test
    df = pd.DataFrame(model.predict_proba(x_test))
    preds = df.drop(columns=0)
    
    # 
    fpr, tpr, _ = roc_curve(y_test, preds)
    auc = roc_auc_score(y_test, preds)

    df2 = pd.DataFrame(model.predict_proba(x_train))
    preds2 = df2.drop(columns=0)
    fpr2, tpr2, _ = roc_curve(y_train, preds2)
    auc2 = roc_auc_score(y_train, preds2)
    
    with sns.axes_style(style="darkgrid"):
        plt.plot(fpr,tpr,color='orange',label="%s test, auc=%s" %(model_name, format(auc,".3f")))
        plt.plot(fpr2,tpr2,color='blue',label="%s train, auc=%s" %(model_name, format(auc2,".3f")))
        plt.plot(y_test,y_test,color='gray',linestyle=':') #baseline 50%
    plt.legend(loc='lower right')    
    
    sns.despine(left=True,bottom=True)

In [10]:
preprocess_pipe = make_pipeline(MinMaxScaler())

In [11]:
#create pipeline
lr_pipe = make_pipeline(SMOTE(random_state = 42), 
                        LogisticRegression(max_iter=200, random_state = 42)
                       )
    

# define grid params for logistic regression
lr_params = {
    'logisticregression__C': [0.02, 0.05, 0.1],
    'logisticregression__solver' : ['newton-cg','saga', 'sag', 'lbfgs'],
    'logisticregression__multi_class': ['multinomial', 'ovr', 'auto'],
    'logisticregression__penalty':['l1','l2','elasticnet'],
    'logisticregression__l1_ratio':[None, 0.25,0.5,0.75],
    'logisticregression__class_weight':[None, 'balanced']
}

# Instantiate GridSearchCV
# Due to imbalanced data, scoring = roc & auc
grid_lr = GridSearchCV(lr_pipe, param_grid = lr_params, verbose = 1, n_jobs=-1, cv=3, scoring='roc_auc')

In [12]:
x_train_t = preprocess_pipe.fit_transform(x_train)
x_test_t = preprocess_pipe.transform(x_test)

In [ ]:
grid_lr.fit(x_train_t,y_train)

Fitting 3 folds for each of 864 candidates, totalling 2592 fits


In [ ]:
print("best score", grid_lr.best_score_)
print("best params", grid_lr.best_params_)

In [ ]:
score(grid_lr, x_train_t, x_test_t)

In [ ]:
roc(grid_lr,"LogisticRegression", x_train_t, x_test_t)

In [ ]:
##random forest

In [ ]:
# create modelling pipeline with SMOTE to deal with imbalanced classes

rf_pipe = make_pipeline(SMOTE(random_state=42),
                           
                           ### Insert Model here for future modelling ###
                           
                           RandomForestClassifier(random_state=42),
                          )

# define grid params for random forest classifier
rf_params = {
    'randomforestclassifier__n_estimators':[20,50,100],
    'randomforestclassifier__max_depth':[5,20,50],
    'randomforestclassifier__criterion':['gini','entropy'],
    'randomforestclassifier__max_leaf_nodes':[5,10,15],
    'randomforestclassifier__class_weight':['balanced','balanced_subsample'],
}

# Instantiate GridSearchCV
# Due to imbalanced data, scoring = roc & auc
grid_rf = GridSearchCV(rf_pipe, param_grid = rf_params, verbose = 1, n_jobs=-1, cv=5, scoring='roc_auc')

In [ ]:
grid_rf.fit(x_train_t,y_train)

In [ ]:
print("best score", grid_rf.best_score_)
print("best params", grid_rf.best_params_)

In [ ]:
score(grid_rf, x_train_t, x_test_t)

In [ ]:
roc(grid_rf,"RandomForestClassifier", x_train_t, x_test_t)

In [ ]:
## Gradient Boost

In [ ]:
# create modelling pipeline with SMOTE to deal with imbalanced classes

gb_pipe = make_pipeline(SMOTE(random_state=42),
                           
                           ### Insert Model here for future modelling ###
                           
                           GradientBoostingClassifier(random_state=42,
                                                      n_iter_no_change=3)
                          )


# define grid params for gradient boosting
gb_params = {
    'gradientboostingclassifier__learning_rate': [0.1, 0.2, 0.3],
    'gradientboostingclassifier__criterion' : ['friedman_mse','mse'],
    'gradientboostingclassifier__n_estimators': [20,50,100],
    'gradientboostingclassifier__max_features':['sqrt','log2'],
    'gradientboostingclassifier__loss':['deviance','exponential']
}

        
# Instantiate GridSearchCV
# Due to imbalanced data, scoring = roc & auc
grid_gb = GridSearchCV(gb_pipe, param_grid = gb_params, verbose = 1, n_jobs=-1, cv=5, scoring='roc_auc')

In [ ]:
grid_gb.fit(x_train_t,y_train)

In [ ]:
print("best score", grid_gb.best_score_)
print("best params", grid_gb.best_params_)

In [ ]:
score(grid_gb, x_train_t, x_test_t)

In [ ]:
roc(grid_gb,"GradientBoostClassifier", x_train_t, x_test_t)